# Exploratory Analysis of Stock Returns

This notebook explores the statistical properties of stock returns and their stylized facts relevant to volatility modeling.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from data.fetcher import fetch_stock_data, fetch_multiple_stocks
from data.preprocessing import calculate_returns, calculate_realized_variance, clean_data
import config

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 1. Data Loading

In [ ]:
# Fetch data for configured tickers
prices = fetch_multiple_stocks(
    tickers=config.TICKERS,
    start_date=config.START_DATE,
    end_date=config.END_DATE
)

print(f"Data shape: {prices.shape}")
print(f"Date range: {prices.index[0]} to {prices.index[-1]}")
prices.head()

In [ ]:
# Calculate log returns
returns = calculate_returns(prices, method='log')
returns.head()

## 2. Price and Return Visualization

In [ ]:
# Normalize prices to starting value of 100
normalized_prices = prices / prices.iloc[0] * 100

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Price plot
normalized_prices.plot(ax=axes[0], alpha=0.8)
axes[0].set_title('Normalized Stock Prices (Base=100)')
axes[0].set_xlabel('')
axes[0].set_ylabel('Price')
axes[0].legend(loc='upper left')

# Returns plot (using SPY as example)
ticker = 'SPY' if 'SPY' in returns.columns else returns.columns[0]
returns[ticker].plot(ax=axes[1], alpha=0.7, color='steelblue')
axes[1].set_title(f'{ticker} Daily Log Returns')
axes[1].set_ylabel('Return')
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 3. Return Distribution Analysis

In [ ]:
# Summary statistics
summary_stats = pd.DataFrame({
    'Mean': returns.mean() * 252,  # Annualized
    'Std': returns.std() * np.sqrt(252),  # Annualized
    'Skewness': returns.skew(),
    'Kurtosis': returns.kurtosis(),
    'Min': returns.min(),
    'Max': returns.max()
})

summary_stats

In [ ]:
# Distribution plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, ticker in enumerate(returns.columns[:6]):
    ax = axes[i]
    data = returns[ticker].dropna()
    
    # Histogram
    ax.hist(data, bins=50, density=True, alpha=0.7, color='steelblue', edgecolor='white')
    
    # Normal distribution overlay
    x = np.linspace(data.min(), data.max(), 100)
    ax.plot(x, stats.norm.pdf(x, data.mean(), data.std()), 
            'r-', linewidth=2, label='Normal')
    
    ax.set_title(f'{ticker} Return Distribution')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# QQ plots to check normality
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, ticker in enumerate(returns.columns[:6]):
    ax = axes[i]
    data = returns[ticker].dropna()
    stats.probplot(data, dist="norm", plot=ax)
    ax.set_title(f'{ticker} Q-Q Plot')

plt.tight_layout()
plt.show()

## 4. Volatility Clustering

In [ ]:
# Rolling volatility
ticker = 'SPY' if 'SPY' in returns.columns else returns.columns[0]
rolling_vol = returns[ticker].rolling(window=21).std() * np.sqrt(252)

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Squared returns (variance proxy)
axes[0].plot(returns.index, (returns[ticker] ** 2), alpha=0.7, color='steelblue')
axes[0].set_title(f'{ticker} Squared Returns (Variance Proxy)')
axes[0].set_ylabel('Squared Return')

# Rolling volatility
axes[1].plot(rolling_vol.index, rolling_vol, color='darkred', linewidth=1.5)
axes[1].set_title(f'{ticker} 21-Day Rolling Volatility (Annualized)')
axes[1].set_ylabel('Volatility')

plt.tight_layout()
plt.show()

## 5. Autocorrelation Analysis

In [ ]:
from pandas.plotting import autocorrelation_plot

ticker = 'SPY' if 'SPY' in returns.columns else returns.columns[0]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# ACF of returns
pd.plotting.autocorrelation_plot(returns[ticker].dropna(), ax=axes[0, 0])
axes[0, 0].set_title(f'{ticker} Return Autocorrelation')
axes[0, 0].set_xlim([0, 50])

# ACF of squared returns
pd.plotting.autocorrelation_plot((returns[ticker] ** 2).dropna(), ax=axes[0, 1])
axes[0, 1].set_title(f'{ticker} Squared Return Autocorrelation')
axes[0, 1].set_xlim([0, 50])

# ACF of absolute returns
pd.plotting.autocorrelation_plot(returns[ticker].abs().dropna(), ax=axes[1, 0])
axes[1, 0].set_title(f'{ticker} Absolute Return Autocorrelation')
axes[1, 0].set_xlim([0, 50])

# Correlation between returns and lagged squared returns (leverage effect)
lags = range(1, 21)
corrs = [returns[ticker].corr((returns[ticker] ** 2).shift(lag)) for lag in lags]
axes[1, 1].bar(lags, corrs, color='steelblue', alpha=0.7)
axes[1, 1].axhline(y=0, color='red', linestyle='--')
axes[1, 1].set_title('Correlation: Return vs Lagged Squared Return')
axes[1, 1].set_xlabel('Lag')
axes[1, 1].set_ylabel('Correlation')

plt.tight_layout()
plt.show()

## 6. Stylized Facts Summary

Key observations from the analysis:

1. **Fat Tails**: Return distributions have heavier tails than normal (kurtosis > 3)
2. **Negative Skewness**: Returns tend to be negatively skewed
3. **Volatility Clustering**: Large returns tend to be followed by large returns
4. **No Return Autocorrelation**: Returns show little autocorrelation
5. **Squared Return Autocorrelation**: Squared returns are highly autocorrelated
6. **Leverage Effect**: Negative returns often lead to higher future volatility

In [ ]:
# Jarque-Bera test for normality
print("Jarque-Bera Normality Test Results:")
print("="*50)
for ticker in returns.columns:
    data = returns[ticker].dropna()
    stat, p_value = stats.jarque_bera(data)
    print(f"{ticker}: JB Statistic = {stat:.2f}, p-value = {p_value:.4e}")

In [ ]:
# ARCH effect test (Ljung-Box on squared returns)
from statsmodels.stats.diagnostic import acorr_ljungbox

print("\nLjung-Box Test for ARCH Effects (Squared Returns):")
print("="*50)
for ticker in returns.columns:
    squared_returns = (returns[ticker] ** 2).dropna()
    result = acorr_ljungbox(squared_returns, lags=[10], return_df=True)
    lb_stat = result['lb_stat'].values[0]
    lb_pval = result['lb_pvalue'].values[0]
    print(f"{ticker}: LB(10) = {lb_stat:.2f}, p-value = {lb_pval:.4e}")